# Goals 
- Load [PyBEL](https://github.com/pybel/pybel) parser (based on [PyParsing](http://pyparsing.wikispaces.com/HowToUsePyparsing)) with given valid namespaces
- "compile" it
- Pickle and save for later

`BelParser` builds a language in `BelParser.__init__` using some of the arguments to add parse actions that do semantic checks and build a `networkx` graph. It is stored in `BelParser.language`. `BelParser.parseString` is a convenience function wrapping `BelParser.language.parseString`

In [1]:
import os
import time
import json

from pybel.parser import BelParser
import networkx as nx
import dill as pickle

In [2]:
%%time
parser = BelParser(valid_namespaces={
    "A": set('abcdef'),
    "B": set('uvwxyz')
})

CPU times: user 417 ms, sys: 9.34 ms, total: 426 ms
Wall time: 425 ms


In [3]:
%%time
res = parser.language.streamline()

CPU times: user 4.8 s, sys: 161 ms, total: 4.96 s
Wall time: 4.98 s


In [4]:
%time res = parser.parseString('g(B:u) -> g(B:v)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 4.57 ms, sys: 396 µs, total: 4.96 ms
Wall time: 4.98 ms
{
  "subject": {
    "function": "Gene",
    "identifier": {
      "name": "u",
      "namespace": "B"
    }
  },
  "relation": "increases",
  "object": {
    "function": "Gene",
    "identifier": {
      "name": "v",
      "namespace": "B"
    }
  }
}


In [5]:
parser.graph.edges()

[(('Gene', 'B', 'u'), ('Gene', 'B', 'v'))]

In [6]:
path = os.path.expanduser('~/Desktop/parser.pickle')

In [7]:
%%time
with open(path, 'wb') as f:
    pickle.dump(parser, f)

CPU times: user 484 ms, sys: 498 ms, total: 982 ms
Wall time: 1.23 s


In [8]:
print("{:.3f} mb".format(os.path.getsize(path) / 1000000))

207.141 mb


In [9]:
del parser

In [10]:
%%time
with open(path, 'rb') as f:
    parser_reloaded = pickle.load(f) 

CPU times: user 203 ms, sys: 194 ms, total: 397 ms
Wall time: 395 ms


In [11]:
%time res = parser_reloaded.parseString('g(B:v) -> g(B:y)')

CPU times: user 4.21 ms, sys: 606 µs, total: 4.82 ms
Wall time: 4.41 ms


In [12]:
parser_reloaded.graph.edges()

[(('Gene', 'B', 'v'), ('Gene', 'B', 'y')),
 (('Gene', 'B', 'u'), ('Gene', 'B', 'v'))]

In [13]:
del parser_reloaded